# 0.1을 2진법으로 표현하기 #

### 소수점 이하를 2진수로 바꿀 때 ###

컴퓨터가 계산을 틀린 이유는 10진수의 소수중에 2진수로는 바꿀 수 없는 것이 있기 때문이다. 10진수 0.1은 2진수로 절대로 정확하게 바꿀 수 없다. 짧게 소수점 이하 4자리만 생각해보면, 2진수로 나타내는 범위는  0.0000 ~ 0.1111 까지가 되는데 10진수에 대응되는 0.5, 0.25, 0.125, 0.0625 의 4개의 값의 조합만으로 밖에 표시할 수 없다는 뜻이다. 이것만으로 보면 0과 0.0625 사이의 값은 표현이 불가능 하다는 말이다. 또한 0.0625 다음에 바로 0.125 가 나오므로 0.1을 딱 만드는것이 불가능 하게 되어 버린다.

In [6]:
import math

format(math.pi, '.12g')

'3.14159265359'

In [7]:
format(math.pi, '.2f')

'3.14'

In [8]:
repr(math.pi)

'3.141592653589793'

이것이, 진정한 의미에서, 환영임을 깨닫는 것이 중요하다. 단순히 진짜 기곗값의 표시 를 반올림하고 있다. 예를 들어, 0.1은 정확히 1/10이 아니므로, 0.1의 세 개를 합한 것 역시 정확히 0.3이 아니다.

In [9]:
.1 + .1 + .1 == .3

False

이는 파이썬을 포함한 많은 언어들이 메모리 관리 차원에서 부동 소수점을 사용하고 있기 때문이다.

![파이썬](https://steemitimages.com/DQme3vRe1nGigGs1GfZkU5ffbufAs1gSNT4MKqR7F1PcxCi/IEEE754.png)

부동 소수점을 표현하는 방식도 정하는 방식에 따라 다를 수 있지만 일반적으로 사용하고 있는 방식은 IEEE에서 표준으로 제안한 방식이다. 우선 고정 소수점으로 나타낸 263.1을 2진수 부동 소수점 방식으로 변환해 보면 100000111.0.0011001100110011001100... 으로 표현되던 것을 맨 앞에 있는 1 바로 뒤로 소수점을 옮겨서 표현하도록 변환한다. 그러면 1.000001110.0011001100110011001100... * 2^8(2의 8승) 으로 표현 된다.

1. 2^8의 8을 지수라고 하고 하늘색 부분에 기록한다. (IEEE 754 표현 방식에서는 127 + 지수를 기록합니다.)
2. 소수점 이후 숫자열 전체를 가수라고 하고 연두색 부분에 기록한다.

이 방식에 따라서 263.1을 기록하면

- 부호 비트(1 bit) : 0 (양수)
- 지수 비트(8 bit) : 10000111 (127 + 8 = 135)
- 가수 비트(23 bit) : 0.0011001100110011001100
이렇게 표현할 수 있다.
하지만! 여기서도 0.0011001100110011001100은 정확히 0.1을 나타낼 수는 없다.

## 그렇다면 어떻게 컴퓨터는 저장하고 표현할까?##

거의 모든 기계는 IEEE-754 부동 소수점 산술을 사용하고, 거의 모든 플랫폼은 파이썬 float를 IEEE-754 "배정밀도"에 매핑한다. 754 배정밀도는 53비트의 정밀도가 포함되어 있어서, 입력 시 컴퓨터는 0.1을 J/2^N 형태의 가장 가까운 분수로 변환하려고 노력합니다. 여기서 J 는 정확히 53비트를 포함하는 정수다.

In [ ]:
1 / 10 ~= J / (2**N)

를

In [ ]:
J ~= 2**N / 10

로 다시 쓰고, J 가 정확히 53 비트(>= 2^52 이지만 < 2^^53)임을 고려하면, N 의 최적값은 56이다.

In [12]:
2**52 <=  2**56 // 10  < 2**53

True

즉, 56은 J 가 정확히 53비트가 되도록 만드는 N 의 유일한 값이다. J 의 가능한 값 중 가장 좋은 것은 반올림한 몫이다.

In [20]:
q, r = divmod(2**56, 10)
r

6

나머지가 10의 절반보다 크므로, 가장 가까운 근삿값은 올림 해서 얻어진다.

In [14]:
q+1

7205759403792794

따라서 754 배정밀도로 1/10 에 가장 가까운 근삿값은 다음과 같다.

In [15]:
7205759403792794 / 2 ** 56

0.1

분자와 분모를 둘로 나누면 다음과 같이 약분된다.

In [16]:
3602879701896397 / 2 ** 55

0.1

올림을 했기 때문에, 이것은 실제로 1/10 보다 약간 크다는 것에 유의해야한다. 내림을 했다면, 몫이 1/10 보다 약간 작아졌을 것이다. 그러나 어떤 경우에도 정확하게 1/10일 수는 없다.

따라서 컴퓨터는 결코 1/10을 "보지" 못한다. 볼 수 있는 것은 위에서 주어진 정확한 분수, 얻을 수 있는 최선의 754 배정밀도 근삿값이다.

In [17]:
0.1 * 2 ** 55

3602879701896397.0

그 분수에 10^55를 곱하면, 55개의 십진 숫자를 볼 수 있다.

In [18]:
3602879701896397 * 10 ** 55 // 2 ** 55

1000000000000000055511151231257827021181583404541015625

이는 컴퓨터에 저장된 정확한 숫자가 십진수 0.1000000000000000055511151231257827021181583404541015625 와 같음을 의미합니다. 전체 십진법 값을 표시하는 대신, 많은 언어(이전 버전의 파이썬 포함)는 결과를 17개의 유효숫자로 반올림한다.

In [19]:
format(0.1, '.17f')

'0.10000000000000001'